# Dependencies and imports

In [1]:
# Requirements.
%pip install "build>=1.0" "torch>=2.0" "transformers" "../../libs/buildlib" "pytest" "aiohttp"

Processing /home/lmerrick/Code/embed_text_container_service/libs/buildlib
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for buildlib: filename=buildlib-1.0.0-py3-none-any.whl size=3621 sha256=e3531c2d63d2bd5bba10c0694bd56ab054e48be978d608424903e0d661b8669c
  Stored in directory: /tmp/pip-ephem-wheel-cache-cm0jh1ll/wheels/c5/f3/ea/faa799a07226ed5f08d2769708ad11b5735aa5e19906b6c1c9
Successfully built buildlib
  Attempting uninstall: buildlib
    Found existing installation: buildlib 1.0.0
    Uninstalling buildlib-1.0.0:
      Successfully uninstalled buildlib-1.0.0
Note: you may need to restart the kernel to use updated packages.


In [2]:
import shutil
from pathlib import Path

import buildlib as buildlib
import pytest
from build import ProjectBuilder
from transformers.models.bert.modeling_bert import BertModel
from transformers.models.bert.tokenization_bert_fast import BertTokenizerFast


/home/lmerrick/miniconda3/envs/embed-standalone/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Introduction

In order to create a custom text embedding service, we need to create a Docker image, push it to Snowflake, and tell Snowflake to deploy it. In addition to building, pushing, and deploying, this notebook will also cover how to test your Docker image locally to speed up the develoment cycle.

# Building

In order to build an image for our custom service, we put custom data (a `data/` directory) and logic (an `embed.py`, plus a `requirements.txt` specifying any dependencies) into the `build/` directory, and then we use the `buildlib` to run the build process (`buildlib.build()`).

## Clear out `build/`

Let's start with a clean slate by deleting and recreating an empty `build/`.

In [3]:
BUILD_DIR = Path(".").resolve().parents[1] / "build"
shutil.rmtree(BUILD_DIR)
BUILD_DIR.mkdir()

## Prepare the data

For this example, we will use the [`e5-base-v2`](https://huggingface.co/intfloat/e5-base-v2) model from Microsoft. We'll demonstrate how to preload the model weights directly into our Docker image for simpler deployment.

Rather than just using the [`transformers`](https://pypi.org/project/transformers/) library directly, we'll also demonstrate how to include a custom library into the Docker image by building a [pure Python wheel](https://packaging.python.org/en/latest/guides/distributing-packages-using-setuptools/#pure-python-wheels) and including that in the `data/` alongside the model weights. The library we demonstrate on is `embed_lib/`, a small example library we wrote to call the E5 model via `transformers`.

### Put model weights into `build/data/`

In [4]:
# Downloading the model weights.
MODEL_NAME = "intfloat/e5-base-v2"
DATA_DIR = BUILD_DIR / "data"
TOKENISER_DIR = DATA_DIR / "tokenizer"
MODEL_DIR = DATA_DIR / "model"

print(f"Downloading {MODEL_NAME} and saving tokenizer and weights to {DATA_DIR}")

tokenizer = BertTokenizerFast.from_pretrained(MODEL_NAME)
model = BertModel.from_pretrained(MODEL_NAME)
assert isinstance(model, BertModel)  # This is for typechecking.
tokenizer.save_pretrained(TOKENISER_DIR)
model.save_pretrained(MODEL_DIR)

# Validate that our saved files work by loading from them.
tokenizer = BertTokenizerFast.from_pretrained(TOKENISER_DIR)
model = BertModel.from_pretrained(MODEL_DIR)

### Put packaged code into `build/data/`

In [5]:
# Package our Python package into a wheel in the `data/` directory.
wheel_filename = ProjectBuilder(Path("") / "embed_lib").build(
    distribution="wheel", output_directory=DATA_DIR
)
print(f"Built {wheel_filename}")

running bdist_wheel
running build
running build_py
running egg_info
writing src/embed_lib.egg-info/PKG-INFO
writing dependency_links to src/embed_lib.egg-info/dependency_links.txt
writing requirements to src/embed_lib.egg-info/requires.txt
writing top-level names to src/embed_lib.egg-info/top_level.txt
reading manifest file 'src/embed_lib.egg-info/SOURCES.txt'
writing manifest file 'src/embed_lib.egg-info/SOURCES.txt'
installing to build/bdist.linux-aarch64/wheel
running install
running install_lib
creating build/bdist.linux-aarch64/wheel
creating build/bdist.linux-aarch64/wheel/embed_lib
copying build/lib/embed_lib/_batch_iter_util.py -> build/bdist.linux-aarch64/wheel/embed_lib
copying build/lib/embed_lib/__init__.py -> build/bdist.linux-aarch64/wheel/embed_lib
copying build/lib/embed_lib/e5.py -> build/bdist.linux-aarch64/wheel/embed_lib
running install_egg_info
Copying src/embed_lib.egg-info to build/bdist.linux-aarch64/wheel/embed_lib-1.0.0-py3.8.egg-info
running install_scripts
c

## Prepare the logic

In order to actually use our custom package, load our model weights, and perform text embedding, we need to implement the core logic of text embedding. This is done by implementing `get_embed_fn()` inside a file called `embed.py`. The function `get_embed_fn` should load model weights and return a function that maps a single input consisting of a `Sequence[str]` into a single 2d `numpy` array of datatype `np.float32`. Below we give an example.

### Implement `get_embed_fn` inside `build/embed.py`

In [6]:
%%writefile ../../build/embed.py
import logging
from typing import Callable
from typing import cast
from typing import Sequence

import embed_lib.e5
import numpy as np
from transformers.models.bert.modeling_bert import BertModel
from transformers.models.bert.tokenization_bert_fast import BertTokenizerFast

MAX_BATCH_SIZE = 4


def get_embed_fn(logger: logging.Logger) -> Callable[[Sequence[str]], np.ndarray]:
    # Load the model into memory.
    logger.info("[get_embed_fn]Loading model from disk to memory")
    tokenizer = BertTokenizerFast.from_pretrained(
        "/root/data/tokenizer", local_files_only=True
    )
    model = cast(
        BertModel, BertModel.from_pretrained("/root/data/model", local_files_only=True)
    )
    e5_model = embed_lib.e5.E5Model(tokenizer, model)

    def _embed(texts: Sequence[str]) -> np.ndarray:
        result_tensor = embed_lib.e5.embed(
            e5_model=e5_model,
            texts=texts,
            batch_size=MAX_BATCH_SIZE,
            normalize=True,
            progress_bar=False,
        )
        result_array = result_tensor.numpy().astype(np.float32)
        return result_array

    return _embed

Writing ../../build/embed.py


### Add a `build/requirements.txt`

We also need to specify the requirements for our embedding logic. During the build, we will populate the `BUILD_ROOT` environment variable, which enables you to include custom packages (like your `embed_lib` wheel) in your `requirements.txt` by absolute filepath.

In [7]:
%%writefile ../../build/requirements.txt

${BUILD_ROOT}/data/embed_lib-1.0.0-py3-none-any.whl

Writing ../../build/requirements.txt


## Configure

In [8]:
%%writefile ../../build/config.py
from service_config import Configuration

USER_CONFIG = Configuration(embedding_dim=768)

Writing ../../build/config.py


## Build!

Now that all the pieces are in place inside `build/`, we can trigger a build via `buildlib`.

In [9]:
# We now have all the pieces we need to build our service!
list(BUILD_DIR.iterdir()) + list(DATA_DIR.iterdir())

[PosixPath('/home/lmerrick/Code/embed_text_container_service/build/requirements.txt'),
 PosixPath('/home/lmerrick/Code/embed_text_container_service/build/embed.py'),
 PosixPath('/home/lmerrick/Code/embed_text_container_service/build/config.py'),
 PosixPath('/home/lmerrick/Code/embed_text_container_service/build/data'),
 PosixPath('/home/lmerrick/Code/embed_text_container_service/build/data/embed_lib-1.0.0-py3-none-any.whl'),
 PosixPath('/home/lmerrick/Code/embed_text_container_service/build/data/model'),
 PosixPath('/home/lmerrick/Code/embed_text_container_service/build/data/tokenizer')]

In [10]:
# If you are building on a Mac with an arm CPU, you may want to build natively
# for local testing (e.g. perf tests will look really slow otherwise), but remember
# to rebuild to amd64 CPU architecture below before deploying.
buildlib.build(build_dir=BUILD_DIR, platform="linux/arm64", tag="latest_arm64")
buildlib.build(build_dir=BUILD_DIR, platform="linux/amd64", tag="latest")

#0 building with "default" instance using docker driver

#1 [internal] load .dockerignore
#1 transferring context: 2B done
#1 DONE 0.0s

#2 [internal] load build definition from Dockerfile
#2 transferring dockerfile: 1.72kB done
#2 DONE 0.0s

#3 [internal] load metadata for docker.io/library/python:3.8.16-bullseye
#3 DONE 0.7s

#4 [ 1/11] FROM docker.io/library/python:3.8.16-bullseye@sha256:e411647c253b75948394a343b13ff32b5674687df0c54187445d12ee9de2b106
#4 DONE 0.0s

#5 [internal] load build context
#5 transferring context: 439.07MB 1.8s done
#5 DONE 1.8s

#6 [ 2/11] WORKDIR /root
#6 CACHED

#7 [ 3/11] COPY ./libs ./libs
#7 CACHED

#8 [ 4/11] COPY ./service_api ./service_api
#8 DONE 0.7s

#9 [ 5/11] COPY ./service_embed_loop ./service_embed_loop
#9 DONE 0.0s

#10 [ 6/11] COPY ./services_common_code ./services_common_code
#10 DONE 0.0s

#11 [ 7/11] RUN python -m venv --copies --prompt api create venv_api     && python -m venv --copies --prompt embed_loop create venv_embed_loop     && .

# Test locally

You can use `pytest` on the tests in `testing/tests` to get a quick check on whether the service is not working as expected.

In [11]:
test_path = BUILD_DIR.parent / "testing" / "tests" / "test_end_to_end.py"
# with buildlib.run_container_context(tag="latest_arm64"):  # NOTE: You can specify the tag.
with buildlib.run_container_context():
    pytest.main([str(test_path)])

============================= test session starts ==============================
platform linux -- Python 3.8.17, pytest-7.4.2, pluggy-1.3.0
rootdir: /home/lmerrick/Code/embed_text_container_service
plugins: anyio-3.6.2
collected 1 item

../../testing/tests/test_end_to_end.py .                                 [100%]

============================== 1 passed in 37.59s ==============================


## Performance testing

You can use the performance testing scripts to see how your configuration works. Keep in mind that [Snowflake has a 30 second timeout](https://docs.snowflake.com/en/sql-reference/external-functions-implementation) that can be triggered if too many big simultaneous requests bog down the service. Setting small batch sizes and a small internal queue size can help mitigate timeout issues. 

In [12]:
# NOTE: If you're on a mac with an ARM CPU, it may make more sense to performance
# test against an ARM build of the image.
buildlib.start_locally(tag="latest_arm64")

In [13]:
!python ../../testing/perf_test_scripts/checking_for_timeouts.py

Embedding 100 rows via 10 simultaneous queries of 2 batches of 5 rows
Rows embedded: 100%|██████████████████████████| 100/100 [01:13<00:00,  1.43it/s]
             batch_time  last_retry_time  retries
query batch                                      
0     0       35.380532        14.125969       42
      1       31.090335        13.436150       35
1     0        4.790592         4.790591        0
      1       16.246887        16.246884        0
2     0        9.604143         9.604142        0
      1       13.940096        13.940094        0
3     0       12.950786        12.950784        0
      1       12.758978        12.758975        0
4     0       69.296374        12.729390      112
      1        4.155452         4.155449        0
5     0       38.154680        14.378661       47
      1       14.826981        14.826978        0
6     0       44.052837        15.230324       57
      1       15.142855        15.142852        0
7     0       15.919474        15.919473        0

In [14]:
buildlib.stop_locally()

# Deploy

Now that we have build our image and made sure it passes local tests, we can deploy our service.

In [16]:
from getpass import getpass
from textwrap import dedent
import snowflake.connector


# Edit these parameters.
connection_params = {
    "account"   : input("Account: "),
    "user"      : input("Username: "),
}

# Establish and configure connection.
connection_params["password"] = getpass(f"Password:")
connection = snowflake.connector.connect(**connection_params)


## Setup prerequisites

To deploy a service, you need to have a compute pool, a database and schema, and an image repository already set up. You also need a role that you can assume which has the permissions necessary to set up the service. Below is an example setup script.

**NOTE:** If you write your embedding code to exploit GPU acceleration, you may want to create a GPU pool. However, for lighweight models and low throughput use-cases (e.g. less than 10QPS of single-row embedding queries over short texts), you may get better price performance from a smaller CPU machine.

In [26]:
compute_pool = "text_embed_gpu"
database = "custom_ml"
schema = "ml"
role = "embed_text_manager"
image_repository = "image_repo"
spec_stage = "service_specs"
setup_sql_commands = dedent(
    f"""
    use role accountadmin;

    -- create a compute pool
    create compute pool if not exists {compute_pool}
    min_nodes = 1
    max_nodes = 1
    instance_family = gpu_3;

    -- create a new database and schema
    create or replace database {database};
    create or replace schema {schema};

    -- give yourself a new role to manage text embedding
    create or replace role {role};
    grant all on database {database} to role {role};
    grant all on schema {database}.{schema} to role {role};
    grant usage on compute pool text_embed_gpu to role {role};
    grant monitor on compute pool text_embed_gpu to role {role};
    grant role embed_text_manager to user {connection_params["user"]};

    -- use the new role to set up the image repo and service spec stage
    use role {role};
    use database {database};
    use schema {schema};
    create or replace image repository {image_repository};
    create or replace stage {spec_stage};
    """
)
buildlib._run_sql(connection, setup_sql_commands)

use role accountadmin;
-- create a compute pool
create compute pool if not exists text_embed_gpu
min_nodes = 1
max_nodes = 1
instance_family = gpu_3;
-- create a new database and schema
create or replace database custom_ml;
create or replace schema ml;
-- give yourself a new role to manage text embedding
create or replace role embed_text_manager;
grant all on database custom_ml to role embed_text_manager;
grant all on schema custom_ml.ml to role embed_text_manager;
grant usage on compute pool text_embed_gpu to role embed_text_manager;
grant monitor on compute pool text_embed_gpu to role embed_text_manager;
grant role embed_text_manager to user admin;
-- use the new role to set up the image repo and service spec stage
use role embed_text_manager;
use database custom_ml;
use schema ml;
create or replace image repository image_repo;
create or replace stage service_specs;


## Push the image

You can use the `buildlib.push` convenience function to push your image. If you're already logged into the image repository in Docker, you can omit the `username` and `password` arguments and pass `skip_login` instead.

In [30]:
image_repository_url = (
    f"{connection_params['account']}.registry.snowflakecomputing.com/"
    f"{database}/{schema}/{image_repository}"
)
buildlib.push(
    repo_url=image_repository_url,
    username=connection_params["user"],
    password=connection_params["password"],
    tag="latest",
    skip_login=False,
)

WARNING! Your password will be stored unencrypted in /home/lmerrick/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



Login Succeeded
Pushing ...registry.snowflakecomputing.com/custom_ml/ml/image_repo/embed_text_service:latest
The push refers to repository [....registry.snowflakecomputing.com/custom_ml/ml/image_repo/embed_text_service]
8b58c7ca5169: Preparing
afaaae059008: Preparing
9395b0d2f52e: Preparing
e7c038323fab: Preparing
33fe9da60550: Preparing
30f6be93e888: Preparing
ce5e3a08a090: Preparing
e048b09748e7: Preparing
81c4730313ac: Preparing
30f6be93e888: Waiting
5f70bf18a086: Preparing
ef9ab794a5c5: Preparing
81c4730313ac: Waiting
e048b09748e7: Waiting
d9fb275baa3d: Preparing
af5d571a6550: Preparing
5f70bf18a086: Waiting
ea9a66bcf3b5: Preparing
d140420135e3: Preparing
b4b4f5c5ff9f: Preparing
b0df24a95c80: Preparing
d9fb275baa3d: Waiting
ef9ab794a5c5: Waiting
af5d571a6550: Waiting
ea9a66bcf3b5: Waiting
974e52a24adf: Preparing
ce5e3a08a090: Waiting
d140420135e3: Waiting
b4b4f5c5ff9f: Waiting
b0df24a95c80: Waiting
974e52a24adf: Waiting
8b58c7ca5169: Layer already exists
33fe9da60550: Layer already

## Create the service

We're almost there! We've built our image and pushed it, now we just need to put together a service spec and run a `create service ...;` statement. Luckily, `buildlib.deploy_service` makes this quite straightforward.

In [31]:
buildlib.deploy_service(
    connection,
    role=role,
    database=database,
    schema=schema,
    spec_stage=spec_stage,
    compute_pool=compute_pool,
    image_repository=image_repository,
    # If for some reason your image repository is in a different database/schema
    # you can specify a separate database/schema, too. Buildlib expects the spec
    # stage to be in the same database/schema as the service, though.
    image_database=database,
    image_schema=schema,
)

use role embed_text_manager;
use database custom_ml;
use schema ml;
Writing to /tmp/tmp8t9fzc5p/embed_text_service.yaml:

spec:
  containers:
    - name: embed-text-service
      image: /custom_ml/ml/image_repo/embed_text_service:latest
      readinessProbe:
        port: 8000
        path: /healthcheck
  endpoint:
    - name: endpoint
      port: 8000


create stage if not exists service_specs;
put file:///tmp/tmp8t9fzc5p/embed_text_service.yaml @service_specs overwrite = true;
drop service if exists embed_text_service;
create service embed_text_service
    in compute pool text_embed_gpu
    from @service_specs
    spec='embed_text_service.yaml'
    min_instances = 1
    max_instances = 1;
create or replace function _embed_to_base64(input string)
    returns string
    service=embed_text_service!endpoint
    max_batch_rows=4
    as '/embed';
create or replace function _unpack_binary_array(B binary)
    returns array
    language javascript
    immutable
    as
    $$
        return Ar

# Success!

Text embedding should now be live! Here's an example of giving all users access to the `embed_text` function and calling this function via a preexisting warehouse called `compute_wh`.

In [48]:
buildlib._run_sql(
    connection,
    "use role accountadmin; "
    "grant usage on database custom_ml to role public; "
    "grant usage on schema custom_ml.ml to role public; "
    "use role embed_text_manager; "
    "grant usage on function custom_ml.ml.embed_text(text) to role public; "
    "use role public; "
    "use warehouse compute_wh;"
)
print(connection.cursor().execute("select embed_text('hello world!');").fetchone()[0])

use role accountadmin;
grant usage on database custom_ml to role public;
grant usage on schema custom_ml.ml to role public;
use role embed_text_manager;
grant usage on function custom_ml.ml.embed_text(text) to role public;
use role public;
use warehouse compute_wh;
[
  3.234522650018334e-03,
  -8.582688868045807e-03,
  -3.768758848309517e-02,
  -3.318526782095432e-03,
  4.481324926018715e-02,
  -3.027019649744034e-02,
  3.205846622586250e-02,
  5.281175300478935e-02,
  -1.082690781913698e-03,
  -1.863022521138191e-02,
  -2.498812414705753e-02,
  4.769056290388107e-02,
  -8.714172244071960e-02,
  1.861651986837387e-02,
  -3.132850676774979e-02,
  6.546445656567812e-03,
  2.404147945344448e-02,
  -8.242245763540268e-03,
  3.759678080677986e-02,
  -2.006381563842297e-02,
  -4.824949055910110e-02,
  -5.238137021660805e-02,
  4.970205575227737e-02,
  -8.793222717940807e-03,
  5.695972125977278e-03,
  1.114103104919195e-02,
  -5.848464556038380e-03,
  1.621723873540759e-03,
  -4.690508171916